# onnxruntime-training, scikit-learn

Simple examples mixing packages.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

In [3]:
%load_ext mlprodict

## Data and first model

In [4]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X, y = data.data, data.target
y /= 100
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
from sklearn.neural_network import MLPRegressor

nn = MLPRegressor(hidden_layer_sizes=(20,), max_iter=2000)
nn.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(20,), max_iter=2000)

In [6]:
from sklearn.metrics import r2_score
r2_score(y_test, nn.predict(X_test))

0.4558040742590157

## Conversion to ONNX

In [7]:
import numpy
from skl2onnx import to_onnx

nn_onnx = to_onnx(nn, X_train[1:].astype(numpy.float32))

%onnxview nn_onnx

## Training with pytorch + ONNX

In [8]:
from onnx.numpy_helper import to_array

weights = [(init.name, to_array(init)) 
           for init in nn_onnx.graph.initializer
           if 'shape' not in init.name]
[w[0] for w in weights]

['coefficient', 'intercepts', 'coefficient1', 'intercepts1']

In [9]:
from deeponnxcustom.onnxtorch import TorchOrtFactory

try:
    fact = TorchOrtFactory(nn_onnx, weights)
except ValueError as e:
    print(e)

List of weights to train must be sorted but is not in [('coefficient', array([[-7.22082239e-03, -1.41002774e-01,  3.70050609e-01,
        -2.36980408e-01,  2.33176742e-02,  2.76906818e-01,
         2.94260740e-01,  7.19446912e-02,  5.56545496e-01,
         4.55525704e-03, -4.02932465e-02,  6.40429463e-03,
         6.75864809e-04, -2.00717244e-02,  4.15303260e-02,
         6.35441542e-02,  2.44001485e-02,  2.97394454e-01,
         3.09448153e-01,  3.05187926e-02],
       [-1.57331243e-07, -1.38778076e-01, -4.29801792e-01,
         1.27669111e-01, -2.06306140e-04, -4.91674334e-01,
        -8.67220163e-01, -3.21711488e-02, -2.74597764e-01,
        -1.27672649e-03, -3.35495849e-03,  2.72700671e-08,
         6.19695129e-10, -4.50221509e-01, -2.42778230e-02,
         2.55909741e-01, -6.35555685e-01, -8.20961475e-01,
        -2.18635842e-01, -9.93478205e-03],
       [ 6.81988662e-03,  8.30241501e-01,  3.48257840e-01,
        -4.19047087e-01,  1.91311949e-04,  6.27074182e-01,
         1.872057

We need to rename the weights in alphabetical order.

In [10]:
from deeponnxcustom.tools.onnx_helper import onnx_rename_weights

onnx_rename_weights(nn_onnx)
weights = [(init.name, to_array(init)) 
           for init in nn_onnx.graph.initializer
           if 'shape' not in init.name]
[w[0] for w in weights]

['I0_coefficient', 'I1_intercepts', 'I2_coefficient1', 'I3_intercepts1']

In [11]:
fact = TorchOrtFactory(nn_onnx, [w[0] for w in weights])

In [12]:
cls = fact.create_class()

In [13]:
from tqdm import tqdm
import torch


def from_numpy(v, device=None, requires_grad=False):
    v = torch.from_numpy(v)
    if device is not None:
        v = v.to(device)
    v.requires_grad_(requires_grad)
    return v


def train_cls(cls, device, X_train, y_train, weights, n_iter=20, learning_rate=1e-2):
    x = from_numpy(X_train.astype(numpy.float32), 
                   requires_grad=True, device=device)
    y = from_numpy(y_train.astype(numpy.float32),
                   requires_grad=True, device=device)

    weights_tch = [(w[0], from_numpy(w[1], requires_grad=True, device=device))
                   for w in weights]
    weights_values = [w[1] for w in weights_tch]

    all_losses = []
    for t in tqdm(range(n_iter)):
        # forward - backward
        y_pred = cls.apply(x, *weights_values)
        loss = (y_pred - y).pow(2).sum()
        loss.backward()

        # update weights
        with torch.no_grad():
            for name, w in weights_tch:
                w -= w.grad * learning_rate
                w.grad.zero_()

        all_losses.append((t, float(loss.detach().numpy())))
    return all_losses, weights_tch


device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
print("device:", device)

# train_losses, final_weights = train_cls(cls, device, X_train, y_test, weights)

device: cpu


In [14]:
from pandas import DataFrame

# df = DataFrame(data=train_losses, columns=['iter', 'train_loss'])
# df.plot(x="iter", y="train_loss", title="Training loss")